# Training Data

### Labeling DataSet

In [65]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import load_img
from sklearn.metrics import classification_report 
import numpy as np
import sklearn
import os 
import shutil 
import random

In [66]:
train_gen = ImageDataGenerator(rescale=1./255)

# Turning every folder name into class label
train_data = train_gen.flow_from_directory(
    "../Datasets/combined-cleaned-dataset",
    target_size=(224,224),
    batch_size=32,
    class_mode= "categorical"
)

print(train_data.class_indices)

Found 18042 images belonging to 13 classes.
{'battery': 0, 'biological': 1, 'brown-glass': 2, 'cardboard': 3, 'clothes': 4, 'glass': 5, 'green-glass': 6, 'metal': 7, 'paper': 8, 'plastic': 9, 'shoes': 10, 'trash': 11, 'white-glass': 12}


In [67]:
# Spliting our dataset into training, validation, and testing 
# we asked chatgpt for help with spliting and combining our 2 dataset into the same categories specifically with using os makedir for the folders and how to use the program shutil 


#requirement and readme do them 

def split_dataset(source_dir, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    random.seed(42)
    
    for class_folder in os.listdir(source_dir):
        class_path = os.path.join(source_dir, class_folder)
        if not os.path.isdir(class_path):
            continue 
        
        images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg','.jpeg','.png'))]
        random.shuffle(images)
    
        total = len(images)
        train_end = int(total * train_ratio)
        val_end = train_end + int(total * val_ratio)
    
        train_images = images[:train_end]
        val_images = images[train_end:val_end]
        test_images = images[val_end:]
    
        train_class_dir = os.path.join(output_dir, 'train', class_folder)
        val_class_dir = os.path.join(output_dir, 'val', class_folder)
        test_class_dir = os.path.join(output_dir,'test',class_folder)
    
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(val_class_dir, exist_ok=True)
        os.makedirs(test_class_dir, exist_ok=True)
    
        for img in train_images:
            shutil.copy2(os.path.join(class_path, img), os.path.join(train_class_dir, img))

        for img in val_images:
            shutil.copy2(os.path.join(class_path, img), os.path.join(val_class_dir, img))

        for img in test_images:
            shutil.copy2(os.path.join(class_path, img), os.path.join(test_class_dir, img))

        print(f" {class_folder}: {len(train_images)} train / {len(val_images)} val / {len(test_images)} test")
    
if __name__ == "__main__":
    source = "../Datasets/combined-cleaned-dataset"   
    destination = "../Notebooks/the_final_sortdown"          
    split_dataset(source, destination, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)
    

 paper: 1150 train / 246 val / 248 test
 green-glass: 440 train / 94 val / 95 test
 clothes: 3727 train / 798 val / 800 test
 metal: 825 train / 176 val / 178 test
 cardboard: 905 train / 194 val / 195 test
 trash: 583 train / 125 val / 126 test
 glass: 350 train / 75 val / 76 test
 biological: 689 train / 147 val / 149 test
 white-glass: 542 train / 116 val / 117 test
 battery: 661 train / 141 val / 143 test
 brown-glass: 424 train / 91 val / 92 test
 plastic: 942 train / 202 val / 203 test
 shoes: 1383 train / 296 val / 298 test


## Data Augmentation

In [68]:
# Data Augmentation
train_gen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    brightness_range = [0.8, 1.2]
)

# training data 
print("Training Data:")
train_data = train_gen.flow_from_directory(
    "the_final_sortdown/train",
    class_mode='categorical',
    target_size=(224, 224),
    batch_size=32,
    shuffle=True
)


val_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

# validation
print("\nValidation Data:")
val_data = val_gen.flow_from_directory(
    "the_final_sortdown/val",
    class_mode='categorical',
    target_size=(224, 224),
    batch_size=32,
    shuffle=True
)

#testing 
print("\nTesting Data:")
test_data = test_gen.flow_from_directory(
    "the_final_sortdown/test",
    class_mode='categorical',
    target_size=(224, 224),
    batch_size=32,
    shuffle=False
)
class_names = list(test_data.class_indices.keys())



Training Data:
Found 16410 images belonging to 13 classes.

Validation Data:
Found 4985 images belonging to 13 classes.

Testing Data:
Found 5047 images belonging to 13 classes.


## Training Convolutional Neural Network

In [90]:
# Transfer Learning MobileNetV2
base_model = MobileNetV2(
    include_top = False,
    weights ='imagenet',
    input_shape = (224,224,3) #images shape 224x224 pixels 
)
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)

num_classes = len(train_data.class_indices)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=Adam(), # optimizers 
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

#model.summary()

In [70]:
# implementing early stopping to avoid to overfitting 
early_stop = EarlyStopping(
    monitor='val_loss',  
    patience=5,          
    restore_best_weights=True  
)

In [71]:
# training the model 
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs = 7, 
    callbacks = [early_stop]
)

/Users/cindychen/.pyenv/versions/3.9.16/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/7
513/513 ━━━━━━━━━━━━━━━━━━━━ 118s 226ms/step - accuracy: 0.7679 - loss: 0.7594 - val_accuracy: 0.8987 - val_loss: 0.3101
Epoch 2/7
513/513 ━━━━━━━━━━━━━━━━━━━━ 119s 232ms/step - accuracy: 0.8897 - loss: 0.3254 - val_accuracy: 0.9163 - val_loss: 0.2405
Epoch 3/7
513/513 ━━━━━━━━━━━━━━━━━━━━ 120s 234ms/step - accuracy: 0.9067 - loss: 0.2612 - val_accuracy: 0.9284 - val_loss: 0.2038
Epoch 4/7
513/513 ━━━━━━━━━━━━━━━━━━━━ 117s 228ms/step - accuracy: 0.9180 - loss: 0.2243 - val_accuracy: 0.9316 - val_loss: 0.1908
Epoch 5/7
513/513 ━━━━━━━━━━━━━━━━━━━━ 120s 234ms/step - accuracy: 0.9259 - loss: 0.2044 - val_accuracy: 0.9388 - val_loss: 0.1775
Epoch 6/7
513/513 ━━━━━━━━━━━━━━━━━━━━ 114s 222ms/step - accuracy: 0.9371 - loss: 0.1742 - val_accuracy: 0.9408 - val_loss: 0.1613
Epoch 7/7
513/513 ━━━━━━━━━━━━━━━━━━━━ 116s 226ms/step - accuracy: 0.9388 - loss: 0.1559 - val_accuracy: 0.9418 - val_loss: 0.1592


## Accuracy 

In [74]:
test_loss, test_acc = model.evaluate(test_data)
print(f"Test accuarcy: {test_acc:.2f}")

158/158 ━━━━━━━━━━━━━━━━━━━━ 22s 140ms/step - accuracy: 0.9509 - loss: 0.1257
Test accuarcy: 0.94


In [75]:
# Adding Precision, Recall, and F1
from sklearn.metrics import classification_report

# Predict
y_pred = model.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get true labels
y_true = test_data.classes

# Print report
print(classification_report(y_true, y_pred_classes, target_names=class_names))


158/158 ━━━━━━━━━━━━━━━━━━━━ 23s 144ms/step
              precision    recall  f1-score   support

     battery       0.98      0.97      0.97       265
  biological       0.99      0.98      0.98       268
 brown-glass       0.80      0.92      0.85       173
   cardboard       0.98      0.94      0.96       360
     clothes       0.99      0.99      0.99      1493
       glass       0.46      0.16      0.23       140
 green-glass       0.86      0.87      0.86       175
       metal       0.97      0.92      0.94       333
       paper       0.96      0.98      0.97       470
     plastic       0.95      0.91      0.93       381
       shoes       0.96      0.99      0.97       545
       trash       0.96      0.94      0.95       235
 white-glass       0.64      0.94      0.76       209

    accuracy                           0.94      5047
   macro avg       0.88      0.88      0.88      5047
weighted avg       0.94      0.94      0.93      5047



## Reinforcement Learning 

In [91]:
import pandas as pd

# Loading the dataset for RL
df = pd.read_csv("../Datasets/Idemat_2025RevA6.csv", encoding="ISO-8859-1")

# Finding the columns names
print(df.columns.tolist())


['.', '.heading', 'Unnamed: 2', '..1', 'Unnamed: 4', 'Process', 'Total', 'Total.1', 'eco-costs of', 'eco-costs of.1', 'eco-costs of.2', 'eco-costs of.3', 'Carbon ', 'Carbon .1', 'CED', 'ReCiPe2016', 'ReCiPe2016.1', 'ReCiPe', 'ReCiPe.1', 'ReCiPe.2', 'ReCiPe.3', 'ReCiPe.4', 'ReCiPe.5', 'EF 3.1', 'EF 3.1.1', 'carbon', 'pollutants', 'water', 'biodiversity', 'resources', 'ClassyFire', 'chemicals,plastics, and wood', 'Note:', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39']


## Cleaning Dataset

In [110]:
# clean data 
df_filtered = df[["Process", "carbon"]].copy()
df_filtered.columns = ["process", "co2_kg"]

# Strip whitespace, convert types
df_filtered["process"] = df_filtered["process"].str.strip()
df_filtered["co2_kg"] = pd.to_numeric(df_filtered["co2_kg"], errors="coerce")

# Drop bad rows
df_filtered.dropna(subset=["process", "co2_kg"], inplace=True)

# Build lookup
co2_lookup = dict(zip(df_filtered["process"], df_filtered["co2_kg"]))

## Implementing Mapping 

In [108]:
import random
import pandas as pd
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from gymnasium import Env
from gymnasium.spaces import Discrete

# mapping items names from dataset to categories from our CNN
cnn_class_to_process = {
    "plastic": [
        "Carbon fiber-reinforced plastic chopped - for reinforcing plastics",
        "Glassfibre - Metals open loop recycling credit dry"
    ],
    "metal": [
        "Zinc (super plastic)",
        "Lead battery cars (39 Wh per kg)", 
        "Aluminium  recycling credit closed loop (79% virgin part trade mix)", 
        "Aluminium  recycling credit closed loop (79% virgin part trade mix)", 
        "Gold  recycling credit closed loop (80% virgin part trade mix)", 
        "Magnesium  recycling credit closed loop (90% virgin part in trade mix)", 
        "Nickel  recycling credit closed loop (66% virgin part in trade mix)", 
        "Platinum  recycling credit  closed loop (88.5% virgin part in trade mix)",
        "Silver  recycling credit closed loop (45% virgin part trade mix)", 
        "Titanium  recycling credit  closed loop (81% virgin part in trade mix)", 
        "Palladium  recycling credit  closed loop (91% virgin part in trade mix)"
        
    ],
    "cardboard": [
        "Corrugated board - unbleached",
        "Corrugated board - bleached"
    ],
    "glass": [
        "Glass - packaging clear",
        "Glass - packaging mixed"
    ],
    "paper": [
        "Paper - office, recycled"
    ],
    "clothes": [
        "Cotton fibres India - rainfed",
        "Cotton fibres Bangladesh - rainfed", 
        "Cotton fibre from USA (without global seatransport)", 
        "Wool from Australia - transported to Rotterdam"
        
    ],
    "shoes": [
        "Leather Vegetable tanning - hides from Europe",
        "Leather Chrome tanning - hides from Argentina"
    ],
    "battery": [
        "AA cell battery (Alkaline)",
        "AA cell battery (Li-ion)",
        "NiCd battery AA-cell",
        "NiCd battery C-cell",
        "Lithium-ion LiCoO2 laptop battery (180 Wh/kg)",
        "Lead battery cars (39 Wh per kg)"
    ],
    "biological": [
        "Baguette white",
        "Bread multigrain",
        "Bread rye" ,
        "Crispbread",
        "White bread hard", 
        "White bread soft", 
        "Zucchini", 
        "Broccoli", 
        "Tomato", 
        "Popcorn",
        "Cake without butter"
        
    ],
    "trash": [
        "Cake with butter",
        "Residual waste - incinerated"
    ],
    "brown-glass": [
        "Beer (bottle)"
        
    ],
    "green-glass": [
        "Beer (bottle)"
    ],
    "white-glass": [
        "Beer (bottle)", 
        "Young gin",
        "Wine rose", 
        "Wine white dry"
    ]
}

#iniliaze empty dictionary for labels
cnn_class_to_co2 = {}

for label, materials in cnn_class_to_process.items():
    if isinstance(materials, list):
        values = [co2_lookup.get(m) for m in materials if m in co2_lookup]
        values = [v for v in values if pd.notna(v)]  # drop NaNs
        if values:
            cnn_class_to_co2[label] = sum(values) / len(values)  # average
        else:
            cnn_class_to_co2[label] = 1.0
    else:
        value = co2_lookup.get(materials, 1.0)
        cnn_class_to_co2[label] = float(value) if pd.notna(value) else 1.0


# mapping different actions for the RL 0 meaning recycle and 3 is compost
correct_actions = {
    "cardboard": 0,   # Recycle
    "clothes": 1,     # Donate
    "battery": 4,     # Hazardous
    "brown-glass":0,  # Recycle
    "glass": 0,       # Recycle
    "green-glass" : 0,# Recycle
    "metal": 0,       # Recycle
    "paper": 0,       # Recycle
    "plastic": 0,     # Recycle
    "shoes": 1,       # Donate
    "white-glass": 0, # Recycle 
    "trash" : 2,      # Landfill
    "biological": 3   # Compost
    
}

#creating environment for RL 
class WasteDisposalEnv(Env):
    def __init__(self, cnn_class_to_co2, correct_actions):
        super(WasteDisposalEnv, self).__init__()
        self.cnn_class_to_co2 = cnn_class_to_co2
        self.correct_actions = correct_actions
        self.classes = list(cnn_class_to_co2.keys())
        self.action_space = Discrete(5)  # [Recycle, Donate, Landfill, Compost, Hazardous]
        self.observation_space = Discrete(len(self.classes))
        self.current_class = None
        
    # Initialize stats tracker
        self.stats = {
            "white-glass_recyclable": 0,
            "white-glass_nonrecyclable": 0
        }
        
 # need reset, step, commonly used for gymnasium environment 
    def reset(self, seed=None, options=None):
        # makes a random waste selection for a new episode 
        super().reset(seed=seed)
        self.current_class = random.choice(self.classes)
          # Return its index as the observation
        obs = self.classes.index(self.current_class)
        return obs,{}

    def step(self, action):
        co2 = self.cnn_class_to_co2[self.current_class]
        
          # correct disposal action for this waste class
        correct_action = self.correct_actions.get(self.current_class, 2) 
          
        # Special case: white-glass may be recyclable or not (adds uncertainty)
         # rewards 
        if self.current_class == "white-glass":
            if random.random() < 0.8:
                correct_action = 0  # Recyclable
                self.stats["white-glass_recyclable"] += 1
            else:
                correct_action = 2  # Non-recyclable
                self.stats["white-glass_nonrecyclable"] += 1

        reward = co2 if action == correct_action else -co2
        #episode ending for each decision 
        terminated = True
        truncated = False
        
        # starts a new episode with a waste item 
        obs, _ = self.reset()
        
         #Return observation, reward
        return obs, reward, terminated, truncated, {
            "class": self.current_class,
            "co2": co2,
            "action": action,
            "correct": correct_action
        }

In [109]:
env = WasteDisposalEnv(cnn_class_to_co2, correct_actions)
check_env(env)

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

model.save("ppo_waste_disposal")

# Test
obs, _ = env.reset()
for _ in range(5):
    action, _ = model.predict(obs)
    obs, reward, terminated, truncated, info = env.step(action)
    print(f"Predicted class: {info['class']}, Action: {action}, Reward: {reward}")
    
    if terminated or truncated:
        obs, _ = env.reset()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -0.768   |
| time/              |          |
|    fps             | 6542     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean          | -0.514     |
| time/                   |            |
|    fps                  | 4326       |
|    iterations           | 2          |
|    time_elapsed         | 0          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.04795645 |
|    clip_fraction        | 0.351      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.58